In [21]:
from serpapi import GoogleSearch
import os
import xml.etree.ElementTree as ET
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import requests

In [22]:
def fetch_content(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Check if the content is PDF
        if 'application/pdf' in response.headers.get('Content-Type', ''):
            return response.content, 'pdf'
        else:
            # return response.text, 'html'

            # Extract text from HTML content using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            text = soup.get_text(separator='\n')  # Separate text with newlines
            return text, 'html'

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch content from {url}. Error: {e}")
        return None, None

In [23]:
def clean_html_and_extract_text(html_text):
    try:
        soup = BeautifulSoup(html_text, 'html.parser')
        text_content = soup.get_text(separator=' ', strip=True)
        
        # Replace multiple consecutive newlines with a single newline
        text_content = re.sub('\n+', '\n', text_content)
        
        result = text_content.split('\n')
        result = [x for x in result if x != '']
        return "\n".join(result)
    except Exception as e:
        print(f"Failed with parser. Error: {e}")
        return None

In [24]:
def text_summarise(content):
    url = "https://www.semrush.com/goodcontent/api/summary-generator/generate-summary/"
    payload = {
        "text": content,
        "format": "paragraph",
        "length_penalty": 0
    }
    # Make the POST request
    response = requests.post(url, json=payload)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_data = response.json()
        return response_data["summary"]
    else:
        return 'None'

In [25]:
def serp(question):
    # results = client.search({
    # 'engine': 'google',
    # 'q': question,
    # 'hl': 'en',
    # 'location': 'Surat, Gujarat',
    # })
    params = {
      "q": question,
      "location": "Surat, Gujarat",
      "hl": "en",
      "google_domain": "google.com",
      "api_key": "2028f74555d2dc51b35e8c6c863d886005a2b243659e74c16092ce40f6b8fe70"
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

In [64]:
directory_path = "D:\Acuration\SERP_Extraction"
file_path = os.path.join(directory_path,'EVify.txt')

In [63]:
questions = ['What is the official website of EVify electric vehicle company?'
'What is the bio of EVify electric vehicle company?',
'Where is the headquarters location of EVify electric vehicle company?',
'What are the products and services of EVify electric vehicle company?',
'What is the unique selling point (USP) of EVify electric vehicle company?',
'What is the value proposition of EVify electric vehicle company?',
'What is the target market of EVify electric vehicle company?',
'What is the market size of EVify electric vehicle company?',
'What is the competitive landscape of EVify electric vehicle company?',
'What are the business models of EVify electric vehicle company?',
'What are the revenue streams of EVify electric vehicle company?',
'What is the pricing model of EVify electric vehicle company?',
'What is the share price of EVify electric vehicle company?', 
'What is the profit margin of EVify electric vehicle company?',
'What is the total user base of EVify electric vehicle company?',
'How many paying customers do of EVify electric vehicle company have?',
'What are the social media platforms of EVify electric vehicle company?',
'How many followers on social media platforms of EVify electric vehicle company',
'What is the funding info of EVify electric vehicle company?',
'What is the vision and mission of EVify electric vehicle company?',
'What are the key capabilities of EVify electric vehicle company?',
'What is the marketing strategy of EVify electric vehicle company?',
'What is the business strategy of EVify electric vehicle company?',
'Does EVify electric vehicle company have any collaborations or partnerships?',
'What is the cash flow of EVify electric vehicle company?']

In [65]:
#for question in questions:
with open(file_path, 'a', encoding='utf-8') as file:
    file.write(f"\n\n\n")
    file.write(f"{question}\n")
    #results=serp(question)
    if "answer_box" in results:    
        answer=results["answer_box"].get('answer')
        file.write(f"\t{answer}\n")
        title=results["answer_box"].get('title')
        file.write(f"\t{title}\n")
        snippet=results["answer_box"].get('snippet')
        file.write(f"\t{snippet}\n")
        url=results["answer_box"].get('link')
        content,type = fetch_content(url)
        text = clean_html_and_extract_text(content)
        summarised_text = text_summarise(text)
        if summarised_text is not None:
            lines = summarised_text.split("\n")
            for line in lines:
                file.write("\t" + line + "\n")
    if "organic_results" in results:
        for i in results["organic_results"]:
            title=i.get('title')
            file.write(f"\t{title}\n")
            snippet=i.get('snippet')
            file.write(f"\t{snippet}\n")                    
            url=i.get('link')
            content,type = fetch_content(url)
            text = clean_html_and_extract_text(content)
            summarised_text = text_summarise(text)
            if summarised_text is not None:
                lines = summarised_text.split("\n")
                for line in lines:
                    file.write("\t" + line + "\n")
    if "related_questions" in results:
        for i in results['related_questions']:
            question=i.get('question')
            file.write(f"\t{question}\n")
            title=i.get('title')
            file.write(f"\t\t{title}\n")
            snippet=i.get('snippet')
            file.write(f"\t\t{snippet}\n")
            url=i.get('link')
            content,type = fetch_content(url)
            text = clean_html_and_extract_text(content)
            summarised_text = text_summarise(text)
            if summarised_text is not None:
                lines = summarised_text.split("\n")
                for line in lines:
                    file.write("\t\t" + line + "\n")

Failed to fetch content from https://www.investing.com/equities/electric-car-company-cash-flow. Error: 403 Client Error: Forbidden for url: https://www.investing.com/equities/electric-car-company-cash-flow
Failed with parser. Error: object of type 'NoneType' has no len()


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [62]:
results

{'search_metadata': {'id': '658de783e0ded4be3991e8cd',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/3a43275811674e69/658de783e0ded4be3991e8cd.json',
  'created_at': '2023-12-28 21:24:19 UTC',
  'processed_at': '2023-12-28 21:24:19 UTC',
  'google_url': 'https://www.google.com/search?q=What+is+the+cash+flow+of+EVify+electric+vehicle+company%3F&oq=What+is+the+cash+flow+of+EVify+electric+vehicle+company%3F&uule=w+CAIQICITU3VyYXQsR3VqYXJhdCxJbmRpYQ&hl=en&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/3a43275811674e69/658de783e0ded4be3991e8cd.html',
  'total_time_taken': 0.62},
 'search_parameters': {'engine': 'google',
  'q': 'What is the cash flow of EVify electric vehicle company?',
  'location_requested': 'Surat, Gujarat',
  'location_used': 'Surat,Gujarat,India',
  'google_domain': 'google.com',
  'hl': 'en',
  'device': 'desktop'},
 'search_information': {'query_displayed': 'What is the cash flow of EVify electric vehicle company